<a href="https://colab.research.google.com/github/LUCID1010/Autorepair_of_DNN/blob/main/automatic_repair_of_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import string
import torch
import numpy as np
import evaluate
from datasets import Dataset, load_dataset  # To load SNLI
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model

# %%
# ---------- Config ----------
MODEL_NAME = "roberta-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
REPAIR_BATCH_SIZE = 16
EPOCHS = 10
LR = 2e-4
MAX_LEN = 256
SEED = 42
OUTPUT_DIR = "./hybrid_autorepair_snli"
os.makedirs(OUTPUT_DIR, exist_ok=True)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(f"Using device: {DEVICE}")

# %%
# ---------- Perturbation utilities ----------
def add_typo(s: str, p=0.05) -> str:
    # Handle NoneType, which can appear in datasets
    if s is None:
        return ""
    chars = list(s)
    for i in range(len(chars)):
        if random.random() < p and chars[i].isalpha():
            chars[i] = random.choice(string.ascii_lowercase)
    return "".join(chars)

def swap_adjacent(s: str, p=0.02) -> str:
    if s is None:
        return ""
    chars = list(s)
    i = 0
    while i < len(chars) - 1:
        if random.random() < p and chars[i].isalpha() and chars[i+1].isalpha():
            chars[i], chars[i+1] = chars[i+1], chars[i]
            i += 2
        else:
            i += 1
    return "".join(chars)

# %%
# ---------- Load a REAL NLI dataset ----------
print("Loading SNLI dataset...")
# Load 2000 for train, 500 for val
snli_dataset = load_dataset("snli", split="train[:2000]")
val_dataset_raw = load_dataset("snli", split="validation[:500]")

# SNLI labels:
train_dataset = snli_dataset.filter(lambda example: example["label"] != -1)
val_dataset = val_dataset_raw.filter(lambda example: example["label"] != -1)

# Rename columns to match your old code
train_dataset = train_dataset.rename_column("premise", "context")
val_dataset = val_dataset.rename_column("premise", "context")

print(f"Training examples: {len(train_dataset)}")
print(f"Validation examples: {len(val_dataset)}")

# %%
# ---------- Create Perturbed Training Set and Clean Validation Set ----------
# Apply perturbations to the training data
def perturb_function(examples):
    examples["context"] = swap_adjacent(add_typo(examples["context"]))
    examples["hypothesis"] = swap_adjacent(add_typo(examples["hypothesis"]))
    return examples

print("Perturbing training data...")
perturbed_train_dataset = train_dataset.map(perturb_function)

# %%
# ---------- Tokenizer & Model ----------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    ignore_mismatched_sizes=True
).to(DEVICE)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["context"],
        examples["hypothesis"],
        truncation=True,
        max_length=MAX_LEN,
        padding=False
    )

print("Tokenizing datasets...")
# Apply tokenization
tokenized_train = perturbed_train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# %%
# ---------- LoRA repair ----------
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value", "key", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
)
model_peft = get_peft_model(model, lora_config)
model_peft.to(DEVICE)
model_peft.print_trainable_parameters()

# %%
# ---------- Metrics Calculation ----------
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# %%
# ---------- Trainer Setup ----------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=REPAIR_BATCH_SIZE,
    per_device_eval_batch_size=REPAIR_BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
)

data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model_peft,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# %%
# Train / repair
print("Starting training...")
trainer.train()
print("Training complete.")

# %%
# --- Final Evaluation ---
print("\n--- Final Evaluation on Clean Validation Set ---")
eval_results = trainer.evaluate(eval_dataset=tokenized_val)

print("\nEvaluation Results:")
print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Loss: {eval_results['eval_loss']:.4f}")

# %%
# Save adapter & tokenizer
model_peft.save_pretrained(os.path.join(OUTPUT_DIR, "lora_adapter"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "tokenizer"))

print(f"\nTraining complete. Adapter saved at: {os.path.join(OUTPUT_DIR, 'lora_adapter')}")

In [ ]:
!pip install evaluate